In [1]:
!pip install overpy

In [63]:
!pip install geopy

In [64]:
from geopy.geocoders import Nominatim

In [16]:
import overpy

# Create an Overpass API object
api = overpy.Overpass()

In [84]:
area_id_offset = 3600000000

In [37]:
def find_poi(lat, lon, radius_meters=1609*5):
    # Define the Overpass query
    query = f"""
    [out:json];
    (
      nwr["tourism"="hotel"](around:{radius_meters}, {lat}, {lon});
    );
    out body;
    >;
    out skel qt;
    """

    # Execute the query
    result = api.query(query)
    print(type(result))

    # Collect results
    stores = []
    for node in result.nodes:
        stores.append({
            "id": node.id,
            "name": node.tags.get("name", "Unnamed"),
            "latitude": node.lat,
            "longitude": node.lon
        })

    return stores

In [21]:
# Example coordinates (Replace with your specific latitude and longitude)
latitude = 47.659240
longitude = -122.319470

In [38]:
# Find liquor stores within a radius
stores = find_poi(latitude, longitude)

<class 'overpy.Result'>


In [ ]:
# Print the results
for store in stores:
    print(f"Id: {store['id']}, Name: {store['name']}, Latitude: {store['latitude']}, Longitude: {store['longitude']}")

In [24]:
store0 = stores[0]

In [31]:
def find_node(node_id):
    # Define the Overpass query
    query = f"""
    [out:json];
    node({node_id});
    out body;
    """

    # Execute the query
    result = api.query(query)
    print(type(result))

    # Collect results
    if len(result.nodes)==0: return None
    node = result.nodes[0]

    return node

In [32]:
store0_node = find_node(store0['id'])

<class 'overpy.Result'>


In [35]:
store0_node

<overpy.Node id=97526535 lat=47.7216444 lon=-122.2952057>

In [40]:
find_node(10593550408)

<class 'overpy.Result'>


<overpy.Node id=10593550408 lat=47.6261451 lon=-122.3468818>

In [79]:
city = "Seattle, Washington"

In [91]:
def find_city_code(city):
  # Geocoding request via Nominatim
  geolocator = Nominatim(user_agent="city_compare")
  geo_results = geolocator.geocode(city, exactly_one=False, limit=3)

  # print(geo_results)
  city_node = None

  for result in geo_results:
    # print(result.address, result.raw.get("osm_type"))
    if result.raw.get("osm_type") == "relation":
        city_node = result
        break
  # print(city_node.raw.get("osm_id"))

  return city_node.raw.get("osm_id")

In [92]:
find_city_code(city)

237385

In [93]:
# https://stackoverflow.com/questions/52236655/how-to-get-a-geocodearea-xxx-query-to-work-in-python-using-overpy
def find_schools_in_loc(loc_id):
  area_id = loc_id + area_id_offset

  query = f"""
  [out:json][timeout:25];
  area({area_id})->.searchArea;
  nwr["amenity"="school"](area.searchArea);
  out body;
  """
  # print(query)

  # Execute the query
  result = api.query(query)

  # Collect results
  schools = []
  for node in result.nodes:
      schools.append({
          "id": node.id,
          "name": node.tags.get("name", "Unnamed"),
          # "latitude": node.lat,
          # "longitude": node.lon
      })

  return schools

In [94]:
city_code = find_city_code(city)
find_schools_in_loc(city_code)

[{'id': 356540545, 'name': 'Mapleleaf School'},
 {'id': 356540551, 'name': 'Hazel Wolf K-8'},
 {'id': 356541630, 'name': 'Epic Life Academy'},
 {'id': 356541632, 'name': 'Christ the King Elementary School'},
 {'id': 356541685, 'name': 'Assumption Saint Bridget School'},
 {'id': 356541772, 'name': 'Saint Edward Parish School'},
 {'id': 356543720, 'name': 'Learning Way School'},
 {'id': 356543721, 'name': 'Seattle Waldorf School - Main Campus'},
 {'id': 356543722, 'name': 'Concordia Lutheran School'},
 {'id': 356543729, 'name': 'Northwest Montessori School'},
 {'id': 356543786, 'name': 'Bjorn Lih Primary School'},
 {'id': 356543789, 'name': 'Northwest Childrens Academy'},
 {'id': 356543871, 'name': 'Dearborn Park International Elementary School'},
 {'id': 356543874, 'name': 'Happy Medium School'},
 {'id': 356543929, 'name': 'Hope Lutheran School'},
 {'id': 356543930, 'name': 'Calvary Luthern School'},
 {'id': 356543933, 'name': 'Westside School'},
 {'id': 356543934, 'name': 'West Seattle